# Table of Contents<a name="toc"></a>
* [The CBC padding oracle](#prob17)
* [Implement CTR, the stream cipher mode](#prob18)

In [2]:
%%capture
!pip install pycryptodome
!pip install numpy
from Crypto.Cipher import AES
import base64
import binascii
import numpy as np
import random
import uuid
from typing import Tuple, List
from enum import Enum, auto


def pkcs7(data: bytes, *, block_size: int = 16) -> bytes:
    padding = block_size - (len(data) % block_size)
    return data + bytes([padding for x in range(padding)])


def xor_encrypt(cipher: bytes, block: bytes) -> bytes:
    cipher_npa = np.frombuffer(cipher, dtype=np.uint8)
    block_npa = np.frombuffer(block, dtype=np.uint8)
    return np.bitwise_xor(cipher_npa, block_npa).tobytes()
xor_decrypt = xor_encrypt


def aes_cbc_decipher(ciphertext: bytes, key: bytes, iv: bytes) -> bytes:
    decipher = AES.new(key, AES.MODE_ECB)
    plaintext = b""
    previous_block = iv
    while len(ciphertext) > 0:
        segment = ciphertext[:len(iv)]
        ciphertext = ciphertext[len(iv):]
        plaintext += xor_decrypt(decipher.decrypt(segment), previous_block)
        previous_block = segment
    return plaintext


def aes_cbc_encipher(plaintext: bytes, key: bytes, iv: bytes) -> bytes:
    encipher = AES.new(key, AES.MODE_ECB)
    ciphertext = b""
    previous_block = iv
    while len(plaintext) > 0:
        segment = plaintext[:len(iv)]
        plaintext = plaintext[len(iv):]
        encoded_block = encipher.encrypt(xor_encrypt(segment, previous_block))
        ciphertext += encoded_block
        previous_block = encoded_block
    return ciphertext


def generate_key(num_bytes: int=16) -> bytes:
    return bytes([random.randint(0,255) for x in range(num_bytes)])
generate_iv = generate_key


def pkcs7_validation(data: bytes) -> bool:
    # check to see if last byte is 0x01
    if data[-1] == 1:
        return True
    
    if data[-1] != 1 and data[-1] == data[-2]:
        pad_value = data[-1]
        for i in range(1, pad_value):
            if data[-1 * i] != data[-1 * (i + 1)]:
                return False
        return True
    
    return False

# The CBC padding oracle<a name="prob17"></a>

In [4]:
_SECRET_KEY = generate_key()

In [3]:
def aes_cbc_mystery_output() -> Tuple[bytes, bytes]:
    """
    returns: ciphertext, iv
    """
    mystery_texts = [
        "MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=",
        "MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=",
        "MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==",
        "MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==",
        "MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl",
        "MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==",
        "MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==",
        "MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=",
        "MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=",
        "MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93"
    ]
    plaintext_b64 = mystery_texts[random.randint(0, len(myster_texts))]
    plaintext = base64.b64decode(plaintext_b64.encode("utf8"))
    
    plaintext_padded = pkcs7(plaintext, block_size=len(_SECRET_KEY))
    iv = generate_iv(len(_SECRET_KEY))
    
    ciphertext = aes_cbc_encypher(plaintext_padded, _SECRET_KEY, iv)
    
    return ciphertext, iv

In [4]:
def aes_cbc_decrypt(ciphertext: bytes, iv: bytes) -> bool:
    plaintext = aes_cbc_decypher(ciphertext, _SECRET_KEY, iv)
    return pkcs7_validation(plaintext)

# Implement CTR, the stream cipher mode<a name="prob18"></a>